# Downloading dataset

In [2]:
!mkdir -p ~/.kaggle

!mv kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# Now, try downloading the dataset again
!kaggle competitions download -c mashtots-dataset

 67% 146M/219M [00:00<00:00, 1.53GB/s]
100% 219M/219M [00:00<00:00, 944MB/s] 


In [4]:
# Unzip the downloaded dataset
!unzip -q mashtots-dataset.zip

# List the contents of the extracted directory to verify
!ls mashtots-dataset

ls: cannot access 'mashtots-dataset': No such file or directory


# Task
Train a ResNet-like model for image classification using the "mashtots-dataset.zip", then make predictions on the "new_test.csv" dataset, saving the results to "submission.csv" and "detailed_predictions.csv", and generating visualizations of the predictions.

## Kaggle Setup and Dataset Download

### Subtask:
Настройте ключ API Kaggle, загрузите 'mashtots-dataset.zip' и распакуйте его содержимое. Убедитесь, что все необходимые файлы датасета доступны.


**Reasoning**:
The previous command `ls mashtots-dataset` failed, suggesting the dataset was not unzipped into a folder with that specific name. To locate the extracted dataset files, I will use `ls -F` to list all files and directories in the current working directory, which will help identify the actual root directory of the dataset.



In [ ]:
!

# List all contents of the current directory to find the extracted dataset
!ls -F

# Based on the output, identify the folder containing 'Train', 'new_test', etc.
# For example, if you see 'Train/' directly, then '.' is the root. If you see 'mashtots-dataset-unzipped/', then that's the root.
# You might need to adjust the 'root' variable in the training code based on this discovery.

detailed_predictions.csv  mashtots_resnet.pth		 sample_data/
mashtots-dataset/	  prediction_distributions.png	 submission.csv
mashtots-dataset.zip	  predictions_visualization.png  Test_sample/


**Reasoning**:
The previous output showed that a directory named `mashtots-dataset/` was created. To fully confirm that the unzipping was successful and the necessary dataset files (`Train/`, `new_test/`, `sample_submission.csv`) are present, I need to list the contents of this specific directory.



In [ ]:
!
!ls -F mashtots-dataset/


new_test/  sample_submission.csv  Train/


## Import Libraries and Define Configuration

### Subtask:
Импортируйте все необходимые библиотеки Python (torch, numpy, pandas, matplotlib, cv2, albumentations и т.д.). Определите глобальные параметры конфигурации, такие как 'IMG_SIZE', 'NUM_CLASSES', 'BATCH_SIZE', 'EPOCHS', 'LR' и 'DEVICE'.


**Reasoning**:
The subtask requires importing several libraries and defining global configuration parameters. This can be accomplished with a single Python code block.



In [5]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random
import shutil

Train test split

In [6]:
train_dir = 'mashtots-dataset/Train'
test_dir = 'mashtots-dataset/Test'

percentage = 20

os.makedirs(test_dir, exist_ok=True)

for class_name in os.listdir(train_dir):
    train_class_path = os.path.join(train_dir, class_name)
    test_class_path = os.path.join(test_dir, class_name)

    if os.path.isdir(train_class_path):
        os.makedirs(test_class_path, exist_ok=True)

        files = os.listdir(train_class_path)
        random.shuffle(files)

        n_files = int(len(files) * (percentage / 100))

        selected_files = files[:n_files]

        for file_name in selected_files:
            src_path = os.path.join(train_class_path, file_name)
            dst_path = os.path.join(test_class_path, file_name)
            shutil.move(src_path, dst_path)

print("Готово! Файлы перемещены.")

Готово! Файлы перемещены.


Config

In [7]:
IMG_SIZE = 64
NUM_CLASSES = 78
BATCH_SIZE = 64
EPOCHS = 30
LR = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Define Model Architecture

### Subtask:
Определите классы 'ResidualBlock' и 'MashtotsResNet' в точности, как они были использованы в скрипте обучения.


**Reasoning**:
The subtask requires defining the `ResidualBlock` and `MashtotsResNet` classes as they were in the training script. I will extract these class definitions from the initial setup code block.



In [8]:
class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

class MashtotsResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = ResidualBlock(1, 64)
        self.layer2 = ResidualBlock(64, 128, stride=2)
        self.layer3 = ResidualBlock(128, 256, stride=2)
        self.layer4 = ResidualBlock(256, 512, stride=2)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, NUM_CLASSES)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool(x).squeeze(-1).squeeze(-1)
        return self.fc(x)

print("Model architecture (ResidualBlock and MashtotsResNet) defined.")

Model architecture (ResidualBlock and MashtotsResNet) defined.


## Prepare Training and Validation Datasets

### Subtask:
Определите класс 'MashtotsDataset'. Загрузите тренировочные данные, разделите их на тренировочный и валидационный наборы, а затем создайте 'DataLoader'ы' для обучения и валидации, включая аугментацию данных для тренировочного набора.


**Reasoning**:
The subtask requires defining the `MashtotsDataset` class, preparing the training and validation datasets by splitting indices, and creating `DataLoader` instances for both, including data augmentation for the training set. This can be done in a single code block by extracting the relevant sections from the original notebook.



In [9]:
class MashtotsDataset(Dataset):
    def __init__(self, root_dir, indices, augment=False):
        self.samples = []
        self.augment = augment

        for folder in os.listdir(root_dir):
            folder_path = os.path.join(root_dir, folder)
            if not os.path.isdir(folder_path):
                continue

            label = int(folder)
            for file in os.listdir(folder_path):
                self.samples.append((os.path.join(folder_path, file), label))

        self.samples = [self.samples[i] for i in indices]

        self.transform = A.Compose([
            A.Rotate(limit=10, p=0.5),
            A.ShiftScaleRotate(0.05, 0.1, 10, p=0.5),
            A.GaussianBlur(blur_limit=3, p=0.3),
            A.RandomBrightnessContrast(0.2, 0.2, p=0.5),
            A.Normalize(mean=(0.5,), std=(0.5,)),
            ToTensorV2()
        ]) if augment else A.Compose([
            A.Normalize(mean=(0.5,), std=(0.5,)),
            ToTensorV2()
        ])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = self.transform(image=img)["image"]
        return img, label


Preaper data

In [17]:
root = "mashtots-dataset/Train"

all_indices = list(range(sum(len(files) for _, _, files in os.walk(root))))
train_idx, val_idx = train_test_split(all_indices, test_size=0.2, random_state=42)

train_ds = MashtotsDataset(root, train_idx, augment=True)
val_ds = MashtotsDataset(root, val_idx, augment=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print("MashtotsDataset class defined and DataLoaders for training and validation prepared.")

MashtotsDataset class defined and DataLoaders for training and validation prepared.


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Architecture of model

In [15]:
class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1, groups=8):
        super().__init__()

        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.gn1 = nn.GroupNorm(groups, out_ch)

        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.gn2 = nn.GroupNorm(groups, out_ch)

        self.shortcut = nn.Identity()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1,
                          stride=stride, bias=False),
                nn.GroupNorm(groups, out_ch)
            )

    def forward(self, x):
        out = F.relu(self.gn1(self.conv1(x)), inplace=True)
        out = self.gn2(self.conv2(out))
        out = out + self.shortcut(x)
        return F.relu(out, inplace=True)


class MashtotsResNet(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super().__init__()

        # stem (очень помогает стабильности)
        self.stem = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.GroupNorm(8, 64),
            nn.ReLU(inplace=True)
        )

        self.layer1 = ResidualBlock(64, 64)
        self.layer2 = ResidualBlock(64, 128, stride=2)
        self.layer3 = ResidualBlock(128, 256, stride=2)
        self.layer4 = ResidualBlock(256, 512, stride=2)

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

# Training

In [18]:
model = MashtotsResNet().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCHS)

for epoch in range(EPOCHS):
    model.train()
    correct, total = 0, 0
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        _, pred = out.max(1)
        correct += pred.eq(y).sum().item()
        total += y.size(0)

    train_acc = 100 * correct / total

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            _, pred = out.max(1)
            correct += pred.eq(y).sum().item()
            total += y.size(0)

    val_acc = 100 * correct / total
    scheduler.step()

    print(f"Epoch [{epoch+1}/{EPOCHS}] | Train: {train_acc:.2f}% | Val: {val_acc:.2f}%")


Epoch [1/30] | Train: 3.90% | Val: 11.26%
Epoch [2/30] | Train: 44.45% | Val: 74.43%
Epoch [3/30] | Train: 78.50% | Val: 85.47%
Epoch [4/30] | Train: 85.58% | Val: 88.62%
Epoch [5/30] | Train: 87.73% | Val: 89.55%
Epoch [6/30] | Train: 89.22% | Val: 91.31%
Epoch [7/30] | Train: 90.43% | Val: 90.66%
Epoch [8/30] | Train: 91.08% | Val: 91.29%
Epoch [9/30] | Train: 91.67% | Val: 92.14%
Epoch [10/30] | Train: 92.20% | Val: 92.60%
Epoch [11/30] | Train: 92.70% | Val: 92.84%
Epoch [12/30] | Train: 93.01% | Val: 92.37%
Epoch [13/30] | Train: 93.32% | Val: 93.03%
Epoch [14/30] | Train: 93.89% | Val: 93.04%
Epoch [15/30] | Train: 94.28% | Val: 93.59%
Epoch [16/30] | Train: 94.49% | Val: 93.74%
Epoch [17/30] | Train: 94.98% | Val: 94.09%
Epoch [18/30] | Train: 95.38% | Val: 94.13%
Epoch [19/30] | Train: 95.72% | Val: 94.37%
Epoch [20/30] | Train: 95.80% | Val: 94.19%
Epoch [21/30] | Train: 96.26% | Val: 94.48%
Epoch [22/30] | Train: 96.47% | Val: 94.51%
Epoch [23/30] | Train: 96.56% | Val: 94.52

Saving Model

In [19]:
torch.save(model.state_dict(), "mashtots_resnet.pth")
print("🔥 Model saved")

🔥 Model saved


# **TESTING**

# Prepearing Data

In [20]:
root = "mashtots-dataset/Test"

all_indices = list(range(sum(len(files) for _, _, files in os.walk(root))))

test_ds = MashtotsDataset(root, all_indices, augment=True)

test_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# Loading model

In [21]:
model = MashtotsResNet().to(DEVICE)
model.load_state_dict(torch.load("mashtots_resnet.pth"))
model.eval()

MashtotsResNet(
  (stem): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): GroupNorm(8, 64, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
  )
  (layer1): ResidualBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (gn1): GroupNorm(8, 64, eps=1e-05, affine=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (gn2): GroupNorm(8, 64, eps=1e-05, affine=True)
    (shortcut): Identity()
  )
  (layer2): ResidualBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (gn1): GroupNorm(8, 128, eps=1e-05, affine=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (gn2): GroupNorm(8, 128, eps=1e-05, affine=True)
    (shortcut): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): GroupNorm(8, 128, e

# Doing predicition

In [27]:
def tta_predict(model, x):
    preds = []

    preds.append(model(x))
    preds.append(model(torch.flip(x, dims=[3])))  # horizontal flip

    return torch.mean(torch.stack(preds), dim=0)


confidences = []
correct_flags = []

with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        out = tta_predict(model, x)
        probs = F.softmax(out, dim=1)
        conf, pred = probs.max(dim=1)

        confidences.extend(conf.cpu().numpy())
        correct_flags.extend((pred == y).cpu().numpy())

confidences = np.array(confidences)
correct_flags = np.array(correct_flags)

print(f"Mean confidence: {confidences.mean():.3f}")
print(f"Correct mean confidence: {confidences[correct_flags].mean():.3f}")
print(f"Wrong mean confidence: {confidences[~correct_flags].mean():.3f}")


Mean confidence: 0.762
Correct mean confidence: 0.825
Wrong mean confidence: 0.582
